In [18]:

import pandas as pd
import numpy as np
import pickle as pkl
import os
from sklearn.metrics.pairwise import rbf_kernel
import pandas as pd
import numpy as np

import time
from sklearn.metrics import accuracy_score, roc_auc_score
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sb
# import visual tools
import matplotlib as mpl
import matplotlib.gridspec as gridspec
%matplotlib inline
import seaborn as sns

plt.style.use('seaborn-talk')
plt.style.use('bmh')

#plt.rcParams['font.family'] = 'DejaVu Sans Mono'
#plt.rcParams['font.size'] = 9.5
plt.rcParams['font.weight'] = 'medium'
#plt.rcParams['figure.figsize'] = 10,7
blue, green, red, purple, gold, teal = sns.color_palette('colorblind', 6)

# import util libs

from tqdm import tqdm, tqdm_notebook
import warnings
warnings.filterwarnings("ignore")

RANDOM_STATE = 777
###
import pystan
from scipy.stats import pearsonr, zscore
from MFDFA import fgn
# Plot settings
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
sb.set()
from tqdm import tqdm

import numpy as np

import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import cm

# from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import minmax_scale
# from sklearn.preprocessing import MaxAbsScaler
# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import RobustScaler
# from sklearn.preprocessing import Normalizer
# from sklearn.preprocessing import QuantileTransformer
# from sklearn.preprocessing import PowerTransformer

import sys
sys.path.insert(0, '/directory/tothe/handshakefile/')

import stylised_facts_data_utilities.createLOB as createLOB

%matplotlib inline
import os
os.getcwd()


'/home/ak/Documents/Research/PaperCode/stylised_facts'

### drives & paths 

In [19]:
passportDrive = '/media/ak/My Passport1/'
dbtFiles = [f for f in os.listdir('/media/ak/My Passport1/') if str('DBT') in f]

In [20]:
def pickle_load(fileLoc):
    pickle_to_file = pkl.load(open(fileLoc, "rb"))
    return pickle_to_file

In [21]:
# DBTFilesFolder = os.path.join(passportDrive,dbtFiles[2])
# csvFiles =os.listdir(DBTFilesFolder)


In [24]:
passportDriveDestination = '/media/ak/My Passport/DBTLOBFiles/'


In [25]:

idxFile = 20
def fileLoc(idxFile):
    activityClocksDestination = os.path.join(passportDriveDestination, 'activityClocks')
    activityClockFiles = os.listdir(activityClocksDestination)
    fileLoc = os.path.join(activityClocksDestination, activityClockFiles[idxFile])
    return fileLoc

In [ ]:

# pickle_to_file = pkl.load(open(fileLoc, "rb"))
# keys = list(pickle_to_file.keys())
# keys

In [ ]:
# originalDF = pickle_to_file[keys[0]]
# TickBarDF = pickle_to_file[keys[1]]
# VolumeBarDF = pickle_to_file[keys[2]]
# DollarVolumeBarDF = pickle_to_file[keys[3]]




In [28]:
microPriceArray =[]
for idxFile in range(0,19):
    microPriceArray.append(pickle_load(fileLoc(idxFile))['TickBarDf'].micro_price.pct_change().dropna())

In [29]:
x_values =pd.DataFrame(microPriceArray).iloc[:,0:1000].values

In [30]:
x_values.shape

(19, 1000)

In [9]:
def returnsPulse(x):
    cumsum = np.cumsum(x)
    y =  np.vstack([cumsum, np.triu( np.expand_dims( cumsum, axis=0 ) - np.outer( np.ones( len( cumsum )  ), cumsum ).transpose() ) ] ) 
    return y

In [10]:
# def returnsPulse(x):
#     cumsum = np.cumsum(x)
#     y = np.triu(np.vstack( [cumsum] + [cumsum - cumsum[i] for i in range(len(cumsum))]))
#     return pd.DataFrame(y)

In [17]:
# y_df =pd.DataFrame(returnsPulse(testY))

In [12]:
def rollingSum(x):
    cumsum = np.cumsum( x )
    result = np.zeros( [len(x) + 1, len( x )])
    result[0, :] =cumsum
    for i in range( len( x )):
        result[ i+1, :] = (cumsum - cumsum[i])
    return np.triu( result )
        

    
def rollingMean(x):
    """
    Compute rolling mean of each bucket shifted by on time in each row
    """
    cumsum = np.cumsum( x )
    result = np.zeros( [len(x + 1), len( x )])
    den = np.arange(1, len( x ) + 1)
    result[0, :] =cumsum/den
    for i in range(len( x)):
        result[ i+1, :] = (cumsum - cumsum[i])/(den - (i +1))
    return np.triu( result)
    
 

In [13]:
def return_tstats_over_days( xs):
    """
    computes the t-statistic over multiple windows across many days
    
    """
    T, N = xs.shape
    result = np.zeros( [N+1, N])
    # compute the cumulative returns for trades with entry at tick at n=0
    cumsum = np.cumsum(xs, axis =1) #of shape [T, N]
    
    result[0, :] = np.mean( cumsum, axis=0)/np.std( cumsum, axis=0)
    for n in range(N):
        ##subtract cumsum of shape [T,N] with cumsum[:,n] of [T,1], to compute the returns for trades with entry at n-th tick
        returns = cumsum - np.expand_dims(cumsum[:, n], axis =1)
        ##then compute the t-statistics over time(first) dimension
        result[n+1, :]= np.mean(returns, axis=0)/np.std(returns, axis=0)
    return np.triu(result)

In [14]:
# x = np.random.normal(size =[10,20000])
# x.shape

In [15]:
t_stats= return_tstats_over_days(x_values)

In [16]:
pd.DataFrame(t_stats)

""
0


In [51]:
return_tstats_over_days(x_values)

array([[-0.23116551, -0.32253977, -0.52147569, ...,  0.32595001,
         0.32984009,  0.32963081],
       [ 0.        , -0.28836333, -0.62769326, ...,  0.33149816,
         0.33538743,  0.33508387],
       [ 0.        ,  0.        , -0.94269921, ...,  0.33608811,
         0.33995954,  0.33956706],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.62511955,  0.34708369],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.18525409],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [54]:
t_stat_df = pd.DataFrame(t_stats)

In [57]:
t_stat_df

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,-0.231166,-0.322540,-0.521476,-0.219908,-0.038699,-0.191159,-0.184622,-0.305734,-0.177124,-0.199659,...,0.328421,0.334895,0.330355,0.333382,0.332560,0.335699,0.319014,0.325950,0.329840,0.329631
1,0.000000,-0.288363,-0.627693,-0.154629,0.046692,-0.142553,-0.142822,-0.296459,-0.154034,-0.178729,...,0.333880,0.340441,0.335817,0.338933,0.338112,0.341234,0.324494,0.331498,0.335387,0.335084
2,0.000000,0.000000,-0.942699,0.030046,0.151171,-0.077849,-0.085544,-0.259232,-0.115547,-0.144256,...,0.338326,0.344963,0.340298,0.343496,0.342687,0.345807,0.329055,0.336088,0.339960,0.339567
3,0.000000,0.000000,0.000000,0.680586,0.370730,0.103821,0.064902,-0.144709,-0.015486,-0.049687,...,0.344058,0.350685,0.345984,0.349216,0.348411,0.351498,0.334816,0.341867,0.345707,0.345241
4,0.000000,0.000000,0.000000,0.000000,0.199524,-0.120311,-0.123027,-0.325380,-0.144970,-0.176971,...,0.339649,0.346335,0.341652,0.344872,0.344067,0.347193,0.330378,0.337448,0.341350,0.340933
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.684796,-0.685734,-0.465785,-0.207667
997,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.269289,0.467540,0.368739
998,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.625120,0.347084
999,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.185254
